In [ ]:
import pandas as pd
import numpy as np
# from pytz import timezone
from datetime import datetime as dt
from datetime import timedelta
from typing import Optional
import pendulum

airline_id_mapping = pd.read_csv('data/airline_id.csv').set_index('two_letter')
# display(airline_id_mapping)
airport_timezone = pd.read_csv('data/airports.csv').set_index('code')['time_zone_id']
aircraft_id = pd.read_csv('data/aircraft_id.csv')


df = pd.read_csv('flightlogs/openflights-backup-2023-10-10.csv', usecols=[
    'Date',
    'From',
    'To',
    'Flight_Number',
    'Airline',
    'Duration', 
    'Seat', 'Seat_Type', 'Class', 'Reason', 'Plane',
       'Registration', 'Note', ], dtype = {
           'Flight_Number': 'str'
       },
       parse_dates=['Date', 'Duration']
       ,encoding='utf-8')
# df['Dep time'] = pd.to_datetime(df['Date']).dt.time
df['Dep time'] = df['Date'].dt.time
df['Duration'] = df['Duration'].dt.time
df['Interval'] = df['Duration'].map(lambda x: timedelta(hours=x.hour, minutes=x.minute, seconds=x.second))
df['Date'] = df['Date'].dt.date

# df.head()
df


In [ ]:
plane_mapping= {
    'Embraer (ERJ) 190': 'Embraer ERJ-190', 
    'Embraer (ERJ) 195': 'Embraer ERJ-195', 
    'BAe 146': 'British Aerospace BAe-146'
}

df['Aircraft'] = df['Plane'].map(lambda x: plane_mapping.get(x, x))
df

In [ ]:
airfract_list = pd.read_json('data/aircraft.json').drop_duplicates()
# airfract_list
df['Aircraft_id']=df['Aircraft'].map(lambda x: airfract_list.set_index('name')['id'].get(x))
df[df['Aircraft_id'].isnull()]['Plane'].drop_duplicates().to_list()

In [ ]:
df.columns

In [ ]:
airline_codes = pd.read_excel('data/Bansard-airlines-codes-IATA-ICAO.xlsx').set_index('Airline Name') #['IATA Designator']
airline_codes.get('IATA Designator', 'Aer Lingus')

In [ ]:
aircraft_id

In [ ]:
airline = df['Airline']
df['Airline_code'] = airline.map(lambda x: airline_codes.get('IATA Designator').get(x))
airline_error_list = df.loc[df['Airline_code'].isna(), 'Airline']


missing_airline_mapping = {
    'TAM Brazilian Airlines' : 'JJ',
    'AZUL': 'AD',
    'Copa Airlines': 'CM',
    'Star Peru (2I)': '2I',
    'Cubana de Aviación' : 'CU',
    'JetBlue Airways': 'B6',
    'Jetstar Airways': 'JQ',
    'Tiger Airways Australia': 'TT',
    'Singapore Airlines': 'SQ',
    'KLM Royal Dutch Airlines': 'KL'
}
airline_error_list = df.loc[df['Airline_code'].isna(), 'Airline'] #.drop_duplicates()
df.loc[airline_error_list.index, 'Airline_code'] = df.loc[airline_error_list.index, 'Airline'].map(lambda x: missing_airline_mapping.get(x))
df['Flight number'] = df['Airline_code'] + df['Flight_Number']
df['Airline_id'] = df['Airline_code'].map(lambda x: airline_id_mapping['Airline_id'].get(x)).fillna(0).astype(int)
df['Airline'] = df['Airline_code'].map(lambda x: airline_id_mapping['Airline'].get(x))



In [ ]:
# Fill in arr time
def calc_arr_time(date: str, from_airport: str, dest_airport:str, dep_time: str, duration):
    from_tz = airport_timezone.get(from_airport)
    dest_tz = airport_timezone.get(dest_airport) 
    from_time = pendulum.parse(date.strftime('%Y-%m-%d')  + ' ' + str(dep_time), tz = from_tz)
    arr_time = from_time + duration
    arr_time_tz = arr_time.in_tz(tz=dest_tz)

    return arr_time_tz.strftime("%H:%M:%S")

df['Arr time'] = df.apply(lambda x: calc_arr_time(x.Date, x.From, x.To, x['Dep time'], x['Interval']), axis=1)
df

In [ ]:

df['Flight class'] = 1
df['Flight reason'] = 1


export_columns = ['Date', 'Flight number', 'From', 'To', 'Dep time', 'Arr time', 'Note',
       'Duration', 'Aircraft', 'Aircraft_id', 'Flight class', 'Flight reason', 'Airline', 'Airline_id']

filename = f"upload/openflight-{dt.now().strftime('%Y-%m-%d')}.csv"
df[export_columns].to_csv(filename, index=False, encoding='utf-8-sig')

